# basic GPT

这个 notebook 将按照 Karpathy 的教程，从零开始实现一个简化版的 GPT 语言模型。

### Topic

- **总体目标**：  
  - 手搓一个最小可用的 GPT（解码器-only Transformer），能在简单文本语料上进行自回归生成。

- **数据与预处理**：  
  - 选择一份小型文本数据集（如 Shakespeare、Linux 源码片段、中文语料等）。  
  - 文本清洗、拆分训练集 / 验证集。  
  - 构建 tokenizer（可以从最简单的字符级 / BPE / WordPiece 开始）。  
  - 将文本转为 token 序列，并按照固定 `block_size` 切成训练样本。

- **模型结构（简化 GPT）**：  
  - Token embedding 与 position embedding 的实现与相加。  
  - **多头自注意力（Multi-Head Self-Attention）**：  
    - 计算 Query / Key / Value，注意力分数与 softmax。  
    - 使用 causal mask（上三角 mask）保证自回归（只能看到过去的 token）。  
    - 多头拆分与拼接，线性投影输出。  
  - 前馈网络（Feed-Forward Network, MLP）层：  
    - 两层线性 + 非线性激活（如 GELU）。  
  - 残差连接与 LayerNorm：  
    - 每个子层（注意力、MLP）前后加上 LayerNorm 和残差。  
  - 多层 Transformer block 堆叠，构成完整 GPT 解码器。  
  - 输出层线性映射到词表维度，得到每个 token 的 logits。

- **训练流程**：  
  - 构建 mini-batch 数据加载（随机采样连续序列）。  
  - 定义交叉熵损失（Cross-Entropy Loss）。  
  - 使用 AdamW 等优化器。  
  - 学习率调度与梯度裁剪（如需要）。  
  - 简单的训练日志与 loss 可视化。

- **推理与采样**：  
  - 给定 prompt，循环自回归生成下一个 token。  
  - 支持 temperature、top-k / top-p 采样策略。  
  - 观察模型生成质量，尝试不同参数的效果。

### Requirements

- **数据需求**：  
  - 一份中小规模纯文本数据集（几 MB ～ 数十 MB 即可）。  
  - 文本编码统一为 UTF-8。  
  - 能够放入内存或通过数据加载器分批读取。

- **计算资源**：  
  - 推荐：支持 CUDA 的 GPU（显存 ≥ 8GB 更舒适），方便更快实验。  
  - 也可在 CPU 上运行，但训练时间会显著变长。  
  - 足够磁盘空间保存数据与模型权重（几十到几百 MB 级别）。

- **软件环境**：  
  - 核心依赖：  
    - `torch`（或 `torch.cuda` 环境，如果有 GPU）  
    - `numpy`  
    - `tqdm`（进度条显示，非必需但推荐）  
    - `matplotlib` 或 `seaborn`（若需要简单可视化）  
  - 可选依赖：  
    - `datasets`（Hugging Face 数据集工具，用于快速加载公开语料）。  
    - `tokenizers`（若想使用更高效的 BPE/WordPiece tokenizer）。 

In [ ]:
'''
1. Data preprocessing
'''
